# AWS Migration: Step-by-Step Implementation Guide

This notebook serves as a checklist and guide for migrating your Breast Cancer Prediction model to AWS. Follow these steps sequentially.

## Prerequisites
*   An active AWS Account.
*   The `breastcancer_env.yaml` file on your local machine.
*   The `clean-data.csv` file on your local machine.

## Step 1: Create S3 Buckets (Storage)
1.  Log in to the **AWS Console** and search for **S3**.
2.  Click **Create bucket**.
3.  **Bucket 1 (Data):** Name it something unique (e.g., `breast-cancer-data-jad-2025`).
    *   Keep defaults (Block Public Access: On).
    *   Click **Create bucket**.
4.  **Bucket 2 (Models):** Name it something unique (e.g., `breast-cancer-models-jad-2025`).
    *   Click **Create bucket**.
5.  **Upload Data:**
    *   Go into your **Data Bucket**.
    *   Click **Upload** -> **Add files**.
    *   Select your local `data/clean-data.csv`.
    *   Click **Upload**.

## Step 2: Create IAM Role (Permissions)
This role allows your EC2 instance to write to your S3 buckets.
1.  Search for **IAM** in the console.
2.  Go to **Roles** -> **Create role**.
3.  **Trusted entity type:** AWS Service.
4.  **Service or use case:** EC2.
5.  Click **Next**.
6.  **Add permissions:** Search for and select `AmazonS3FullAccess`.
    *   *Note: In a strict production environment, you would limit this to just your specific buckets, but FullAccess is fine for setup.*
7.  Click **Next**.
8.  **Role name:** `EC2-S3-Access-Role`.
9.  Click **Create role**.

### Step 3: Launch EC2 Instance (For Training)
We still need an EC2 instance, but now its primary role is **Model Training** and **Data Processing**, not hosting the website.

1.  **Go to EC2 Dashboard** -> Launch Instance.
2.  **Name:** `BreastCancerTraining`.
3.  **OS:** Ubuntu 24.04 LTS.
4.  **Instance Type:** `t3.medium` (Recommended for training) or `t2.micro` (Free tier, might be slow).
5.  **Key Pair:** Create new -> `breast-cancer-key` -> Download `.pem`.
6.  **Network Settings:**
    *   Allow SSH traffic from My IP.
    *   (Optional) Allow HTTP/HTTPS if you want to download things easily.
7.  **Launch Instance**.

## Step 4: Create the Training Script
Create a file named `train.py` on your local machine with the following content. 
**Important:** Replace `YOUR_SOURCE_DATA_BUCKET_NAME` and `YOUR_MODEL_BUCKET_NAME` with the actual names of the buckets you created in Step 1.

In [ ]:

import pandas as pd
import boto3
import joblib
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# --- CONFIGURATION ---
# 1. UPDATE THESE WITH YOUR ACTUAL BUCKET NAMES
SOURCE_BUCKET = 'breast-cancer-cleaneddata' 
MODEL_BUCKET = 'breast-cancer-prediction-models'
DATA_FILE = 'clean-data.csv'
LOCAL_DATA_PATH = 'data/clean-data.csv'

# 1. Load Data
if not os.path.exists(LOCAL_DATA_PATH):
    os.makedirs('data', exist_ok=True)
    print(f"Downloading {DATA_FILE} from S3...")
    try:
        s3 = boto3.client('s3')
        s3.download_file(SOURCE_BUCKET, DATA_FILE, LOCAL_DATA_PATH)
        print("Download successful.")
    except Exception as e:
        print(f"Error downloading from S3: {e}")
        print("Make sure you updated SOURCE_BUCKET and that your EC2 role has S3 permissions.")
        raise

data = pd.read_csv(LOCAL_DATA_PATH)

# 2. Preprocessing (Cleaning)
if 'Unnamed: 0' in data.columns: data.drop('Unnamed: 0', axis=1, inplace=True)
if 'id' in data.columns: data.drop('id', axis=1, inplace=True)
if 'Unnamed: 32' in data.columns: data.drop('Unnamed: 32', axis=1, inplace=True)

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

# Encode Target
le = LabelEncoder()
y = le.fit_transform(y)

# 3. Define Pipeline (StandardScaler -> PCA -> SVM)
pipeline = Pipeline([
    ('scl', StandardScaler()),
    ('pca', PCA(n_components=2)),
    # Best params from Notebook (C=0.1, gamma=0.001, kernel='linear')
    ('clf', SVC(C=0.1, gamma=0.001, kernel='linear', probability=True)) 
])

# 4. Train
print("Training model...")
pipeline.fit(X, y)
print("Training complete.")

# 5. Save and Upload
LOCAL_MODEL_PATH = 'model_v1.pkl'
joblib.dump(pipeline, LOCAL_MODEL_PATH)
print(f"Model saved locally to {LOCAL_MODEL_PATH}")

print("Uploading to S3...")
s3 = boto3.client('s3')
s3.upload_file(LOCAL_MODEL_PATH, MODEL_BUCKET, 'latest_model.pkl')
print(f"Success! Model uploaded to s3://{MODEL_BUCKET}/latest_model.pkl")

### Is this the most efficient model?

**Short Answer:** Yes, this architecture (SVM with Scaling) was the top performer in your analysis. We have updated the script with your specific hyperparameters (`C=0.1`, `kernel='linear'`) based on your Grid Search results.

**Detailed Explanation:**
1.  **Why SVM?**
    In your notebook `NB6 Comparison between different classifiers`, you compared Logistic Regression, LDA, KNN, CART, Naive Bayes, and SVM.
    *   Initially, SVM performed poorly.
    *   **However**, after you applied `StandardScaler` (Standardization), SVM became the **most accurate algorithm**, outperforming the others. This is why we selected `SVC` for your production script.

2.  **Why PCA?**
    Your notebooks (`NB5` and `NB6`) use Principal Component Analysis (PCA).
    *   In `NB6`, you explicitly defined a pipeline: `StandardScaler` -> `PCA(n_components=2)` -> `SVC`.
    *   Using `n_components=2` makes the model extremely **efficient** (fast to train, small to store) because it only looks at the 2 most important variance features instead of all 30.
    *   *Trade-off:* If you want slightly higher accuracy at the cost of speed, you could increase `n_components` to 10 (as explored in `NB5`) or remove PCA entirely. But for a "lightweight" cloud deployment, keeping PCA is a smart move for efficiency.

3.  **Why C=0.1?**
    In `NB5`, you ran a `GridSearchCV` to find the best `C` (Regularization) and `gamma`.
    *   We have updated the `train.py` script to use `C=0.1` and `kernel='linear'` as per your specific results.

**Conclusion:**
The `train.py` script represents the **best architecture** (Scaled SVM) found in your research. It is "efficient" because it balances high accuracy (via SVM) with low computational cost (via PCA).

## Step 5: Deploy to EC2
Now that your infrastructure is ready and your code is written, let's run it.

1.  **Connect to EC2:**
    *   Open your terminal (WSL/Ubuntu).
    *   Run the following command:
    ```bash
    ssh -i "~/breast-cancer-key.pem" ubuntu@18.184.78.242
    ```

2.  **Install Environment Manager (Micromamba):**
    *(You have already completed this step)*
    Run these commands inside the EC2 terminal:
    ```bash
    "${SHELL}" <(curl -L micro.mamba.pm/install.sh)
    # Press Enter to accept defaults
    source ~/.bashrc
    ```

3.  **Upload Files:**
    **CRITICAL:** Do NOT run this in the EC2 terminal. Open a **NEW** terminal window (WSL/Ubuntu) on your local machine.
    
    Use `scp` to copy your files to the server.
    ```bash
    # 1. Upload Environment File
    scp -i "~/breast-cancer-key.pem" "/mnt/c/Users/Jad Zoghaib/OneDrive/Desktop/CC_Breast_Cancer/Breast-cancer-risk-prediction/breastcancer_env.yaml" ubuntu@18.184.78.242:~/
    
    # 2. Upload Training Script
    scp -i "~/breast-cancer-key.pem" "/mnt/c/Users/Jad Zoghaib/OneDrive/Desktop/CC_Breast_Cancer/Breast-cancer-risk-prediction/train.py" ubuntu@18.184.78.242:~/
    ```

4.  **Setup & Run (Back in EC2 Terminal):**
    ```bash
    # Create environment
    micromamba create -f breastcancer_env.yaml
    
    # Activate
    micromamba activate breastcancer
    
    # Install boto3 (if missing)
    micromamba install boto3
    
    # Run the training
    python train.py
    ```

5.  **Verify:**
    Check your S3 bucket "Models". You should see `latest_model.pkl`.

# Phase 1: The Serverless Application (Inference)
We will use **AWS Lambda** for the backend logic (running the model) and **Amazon S3** to host the static website.

### Step 6: Create the Lambda Function
This function will fetch the model from S3, load it, and make a prediction.

1.  **Create IAM Role (Permissions):**
    *   Go to **IAM** -> **Roles** -> **Create role**.
    *   Trusted entity: **AWS Service** -> **Lambda**.
    *   **Add permissions:** Search for and select:
        *   `AmazonS3FullAccess` (Read Model)
        *   `AmazonDynamoDBFullAccess` (Save Records)
        *   `AmazonSNSFullAccess` (Send Emails)
    *   **Role name:** `LambdaMLRole`.
    *   Click **Create role**.

2.  **Create Function:**
    *   Go to **AWS Lambda** -> **Create Function**.
    *   **Name:** `PredictorLambda`.
    *   **Runtime:** **Python 3.10**.
    *   **Architecture:** **x86_64**.
    *   **Permissions:** Use an existing role -> Select `LambdaMLRole`.
    *   **Create Function**.

3.  **Add Layers (Crucial):**
    *   **Layer 1 (Scikit-Learn):** Create a custom layer named `sklearn-light` using the `sklearn_light.zip` generated in `Lambda_Layer_Fix.ipynb`. Add it to the function.
    *   **Layer 2 (Numpy):** Add a layer via **Specify an ARN**: `arn:aws:lambda:eu-central-1:770693421928:layer:Klayers-p310-numpy:16`.

4.  **Code Source:**
    *   Paste the code below into `lambda_function.py`.
    *   **Deploy**.

In [ ]:
import json
import boto3
import joblib
import os
from io import BytesIO

# --- CONFIGURATION ---
BUCKET_NAME = 'breast-cancer-prediction-models'
MODEL_KEY = 'latest_model.pkl'
DYNAMODB_TABLE = 'Patient_Entries'  # Updated to match your existing table
SNS_TOPIC_ARN = 'arn:aws:sns:eu-central-1:469541406278:MalignantAlerts'

s3 = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')
sns = boto3.client('sns')
table = dynamodb.Table(DYNAMODB_TABLE)

def load_model_from_s3():
    print("Loading model from S3...")
    response = s3.get_object(Bucket=BUCKET_NAME, Key=MODEL_KEY)
    model_stream = BytesIO(response['Body'].read())
    model = joblib.load(model_stream)
    print("Model loaded successfully.")
    return model

# Load model globally to reuse across invocations
model = load_model_from_s3()

def process_doctor_feedback(body):
    """Handles the Tick/Cross feedback from the doctor."""
    case_id = body.get('id')
    resolution = body.get('resolution')  # 'confirmed_malignant' or 'confirmed_benign'

    if not case_id or not resolution:
        return {'statusCode': 400, 'body': json.dumps('Missing id or resolution')}

    print(f"Processing feedback for Case {case_id}: {resolution}")

    # Update DynamoDB
    try:
        table.update_item(
            Key={'id': case_id},
            UpdateExpression="SET doctor_resolution = :r, status = :s",
            ExpressionAttributeValues={
                ':r': resolution,
                ':s': 'Resolved'
            }
        )
        return {'statusCode': 200, 'body': json.dumps(f"Case {case_id} resolved as {resolution}")}
    except Exception as e:
        print(f"Error updating DynamoDB: {str(e)}")
        return {'statusCode': 500, 'body': json.dumps(f"Database error: {str(e)}")}

def lambda_handler(event, context):
    try:
        print("Received event:", json.dumps(event))
        
        # Parse body
        if 'body' in event:
            if isinstance(event['body'], str):
                body = json.loads(event['body'])
            else:
                body = event['body']
        else:
            body = event

        # CHECK OPERATION TYPE
        if body.get('operation') == 'feedback':
            return process_doctor_feedback(body)

        # --- PREDICTION LOGIC ---
        # Expecting 'features' list in the body
        features = body.get('features')
        case_id = body.get('id', 'unknown_id')

        if not features:
            return {
                'statusCode': 400,
                'body': json.dumps("Error: 'features' list is required.")
            }

        # Predict directly from list (No Pandas needed)
        # Scikit-learn expects a 2D array: [[f1, f2, ...]]
        prediction = model.predict([features])
        result = 'M' if prediction[0] == 1 else 'B'

        print(f"Prediction for {case_id}: {result}")

        # Save to DynamoDB
        item = {
            'id': case_id,
            'features': str(features),  # Store as string or list
            'prediction': result,
            'status': 'Pending Review'
        }
        table.put_item(Item=item)

        # Send SNS Alert if Malignant
        if result == 'M':
            message = (
                f"URGENT: Malignant Case Detected\n\n"
                f"Case ID: {case_id}\n"
                f"Prediction: Malignant (M)\n"
                f"Status: Pending Doctor Review\n\n"
                f"Please log in to the dashboard to review this case immediately."
            )
            sns.publish(
                TopicArn=SNS_TOPIC_ARN,
                Message=message,
                Subject=f"Alert: Malignant Case {case_id}"
            )

        return {
            'statusCode': 200,
            'body': json.dumps({'prediction': result, 'id': case_id})
        }

    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Internal Server Error: {str(e)}")
        }

ModuleNotFoundError: No module named 'boto3'

### Step 7: Create API Gateway
This allows your HTML page to talk to the Lambda function.

1.  Go to **API Gateway** -> **HTTP API** -> Build.
2.  **API Name:** `BreastCancerAPI`.
3.  **Integrations:** Add Integration -> Lambda -> Choose `PredictorLambda`.
4.  **Configure Routes:**
    *   Method: `POST`
    *   Resource Path: `/predict`
    *   Integration Target: `PredictorLambda`
5.  **Create**.
6.  **CORS Configuration:** (Crucial for web access)
    *   Go to **CORS** in the sidebar.
    *   Access-Control-Allow-Origin: `*`
    *   Access-Control-Allow-Methods: `POST`
    *   Access-Control-Allow-Headers: `content-type`
    *   Save.
7.  **Copy the Invoke URL** (e.g., `https://xyz.execute-api.us-east-1.amazonaws.com`).

### Step 8: Host Frontend on S3
1.  Create a file named `BreastCancerPredictorFrontEnd.html` locally.
2.  **Update the JavaScript fetch URL** to point to your new API Gateway URL + `/predict`.
3.  Go to your S3 Bucket (`breast-cancer-app-frontend`).
4.  **Upload** `BreastCancerPredictorFrontEnd.html`.
5.  Go to **Properties** -> **Static website hosting** -> Enable.
6.  **Index document:** `BreastCancerPredictorFrontEnd.html`.
7.  Save.
8.  **Open the Bucket Website Endpoint** to test your app.

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Breast Cancer Risk System</title>
    <style>
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; max-width: 1000px; margin: 0 auto; padding: 20px; background-color: #f8f9fa; color: #333; }
        .view { display: none; animation: fadeIn 0.5s; }
        .active { display: block; }
        @keyframes fadeIn { from { opacity: 0; } to { opacity: 1; } }
        
        /* Buttons */
        .btn { padding: 12px 24px; margin: 5px; cursor: pointer; border-radius: 6px; font-size: 15px; border: none; transition: all 0.2s; font-weight: 600; }
        .btn:disabled { opacity: 0.6; cursor: not-allowed; }
        .btn-primary { background-color: #007bff; color: white; }
        .btn-primary:hover { background-color: #0056b3; transform: translateY(-1px); }
        .btn-success { background-color: #28a745; color: white; }
        .btn-success:hover { background-color: #218838; }
        .btn-danger { background-color: #dc3545; color: white; }
        .btn-danger:hover { background-color: #c82333; }
        .btn-outline { background: transparent; border: 1px solid #007bff; color: #007bff; }
        .btn-outline:hover { background: #e7f1ff; }

        /* File Upload */
        .upload-area { border: 2px dashed #ccc; padding: 30px; text-align: center; background: white; border-radius: 10px; margin: 20px 0; transition: 0.3s; }
        .upload-area:hover { border-color: #007bff; background: #f0f8ff; }
        
        /* Table */
        .table-container { max-height: 400px; overflow: auto; margin: 20px 0; border: 1px solid #dee2e6; border-radius: 5px; background: white; display: none; }
        table { width: 100%; border-collapse: collapse; font-size: 13px; white-space: nowrap; }
        th, td { padding: 10px; text-align: left; border-bottom: 1px solid #dee2e6; }
        th { background-color: #e9ecef; position: sticky; top: 0; z-index: 10; font-weight: 600; }
        tr:hover { background-color: #f8f9fa; }

        /* Cards */
        .case-card { background: white; border-left: 5px solid #dc3545; border-radius: 5px; padding: 15px; margin: 10px 0; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 8px rgba(0,0,0,0.05); }
        .status-badge { padding: 4px 8px; border-radius: 4px; font-size: 12px; font-weight: bold; }
        .status-malignant { background: #ffebee; color: #c62828; }
        
        /* Logs */
        #analysis-log { background: #2d2d2d; color: #00ff00; padding: 15px; border-radius: 5px; font-family: monospace; max-height: 200px; overflow-y: auto; margin-top: 20px; display: none; }
    </style>
    <script>
        // --- CONFIGURATION ---
        const PREDICT_URL = 'https://qu6y4f29lg.execute-api.eu-central-1.amazonaws.com/predict';

        // --- CONSTANTS ---
        const FEATURE_NAMES = [
            "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", "fractal_dimension_mean",
            "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se", "concave points_se", "symmetry_se", "fractal_dimension_se",
            "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst", "concave points_worst", "symmetry_worst", "fractal_dimension_worst"
        ];

        // --- STATE ---
        let loadedData = []; // Stores objects: { id: string, features: number[] }
        let openCases = [];  // Stores Malignant cases waiting for review

        // --- NAVIGATION ---
        function showView(viewId) {
            document.querySelectorAll('.view').forEach(el => el.classList.remove('active'));
            document.getElementById(viewId).classList.add('active');
            if(viewId === 'cases-view') renderOpenCases();
        }

        // --- FILE HANDLING ---
        function handleFileUpload(event) {
            const file = event.target.files[0];
            if (!file) return;

            const reader = new FileReader();
            reader.onload = function(e) {
                const text = e.target.result;
                parseCSV(text, file.name);
            };
            reader.readAsText(file);
        }

        function parseCSV(csvText, fileName) {
            try {
                // 1. Extract Base ID from Filename
                // Example: "sample_patient_569.csv" -> "569"
                const nameNoExt = fileName.substring(0, fileName.lastIndexOf('.')) || fileName;
                const parts = nameNoExt.split(/[_-\s]+/);
                const baseId = parts[parts.length - 1];

                const lines = csvText.split('\n').filter(line => line.trim() !== '');
                loadedData = [];
                
                // Determine start row (Skip header if present)
                let startRow = 0;
                if (/[a-zA-Z]/.test(lines[0])) {
                    startRow = 1;
                }

                // Parse rows
                const tempRows = [];
                for (let i = startRow; i < Math.min(lines.length, 101); i++) { 
                    const rawValues = lines[i].split(',');
                    
                    // Filter for numeric values (to find features)
                    const numericValues = rawValues
                        .map(v => parseFloat(v))
                        .filter(v => !isNaN(v));

                    // We need at least 30 numeric features
                    if (numericValues.length >= 30) {
                        const features = numericValues.slice(-30);
                        tempRows.push(features);
                    }
                }

                if (tempRows.length === 0) {
                    alert("Could not parse any valid rows with 30 numeric features. Please check your CSV format.");
                    return;
                }

                // Assign IDs based on filename
                loadedData = tempRows.map((features, index) => {
                    // If single row, use the filename ID directly (e.g. "569")
                    // If multiple rows, append index (e.g. "569-1", "569-2")
                    let caseId = baseId;
                    if (tempRows.length > 1) {
                        caseId = `${baseId}-${index + 1}`;
                    }
                    return { id: caseId, features: features };
                });

                renderTable(loadedData);
                document.getElementById('confirm-btn').disabled = false;
                
            } catch (e) {
                console.error(e);
                alert("Error parsing CSV: " + e.message);
            }
        }

        function renderTable(data) {
            const container = document.getElementById('table-container');
            const tableHead = document.getElementById('data-table-head');
            const tableBody = document.getElementById('data-table-body');
            
            // Clear previous
            tableHead.innerHTML = '';
            tableBody.innerHTML = '';

            // Headers
            const headerRow = document.createElement('tr');
            
            // ID Column Header
            const thId = document.createElement('th');
            thId.innerText = "Case ID"; 
            thId.style.minWidth = "80px";
            headerRow.appendChild(thId);
            
            // Feature Headers
            FEATURE_NAMES.forEach(h => {
                const th = document.createElement('th');
                th.innerText = h;
                headerRow.appendChild(th);
            });
            tableHead.appendChild(headerRow);

            // Rows
            data.forEach((item) => {
                const tr = document.createElement('tr');
                
                // ID Cell
                const tdId = document.createElement('td');
                tdId.innerText = item.id; 
                tdId.style.fontWeight = "bold";
                tdId.style.backgroundColor = "#f8f9fa";
                tr.appendChild(tdId);

                // Feature Cells
                item.features.forEach(val => {
                    const td = document.createElement('td');
                    td.innerText = val; 
                    tr.appendChild(td);
                });
                tableBody.appendChild(tr);
            });

            container.style.display = 'block';
        }

        function confirmData() {
            if (loadedData.length > 0) {
                document.getElementById('analyze-btn').disabled = false;
                alert(`Data Confirmed! ${loadedData.length} patient records ready for analysis.`);
            } else {
                alert("No data to confirm. Please upload a valid CSV.");
            }
        }

        // --- LOGIC: BATCH ANALYSIS ---
        async function analyzeRisk() {
            if (!loadedData || loadedData.length === 0) {
                alert("No data loaded. Please upload a CSV first.");
                return;
            }

            showView('analyze-view');
            
            // MIRROR TABLE TO ANALYZE VIEW
            const originalTable = document.getElementById('table-container').innerHTML;
            const targetContainer = document.getElementById('analysis-table-container');
            targetContainer.innerHTML = originalTable;
            targetContainer.style.display = 'block';

            const logDiv = document.getElementById('analysis-log');
            logDiv.style.display = 'block';
            logDiv.innerHTML = "Starting analysis...\n";

            let processed = 0;
            
            // Iterate through loaded data
            for (let i = 0; i < loadedData.length; i++) {
                const item = loadedData[i];
                const features = item.features;
                const patientId = item.id; 

                try {
                    // Call API
                    const response = await fetch(PREDICT_URL, {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({ features: features, id: patientId })
                    });

                    // ERROR HANDLING: Check if the request failed (e.g. 500 or 400)
                    if (!response.ok) {
                        const errorText = await response.text();
                        throw new Error(`Server Error (${response.status}): ${errorText}`);
                    }

                    const result = await response.json();
                    const prediction = result.prediction;

                    // VALIDATION: Check if prediction is missing
                    if (prediction === undefined) {
                        throw new Error("Invalid response: 'prediction' field missing.");
                    }

                    processed++;
                    
                    if (prediction === 'M') {
                        // LOGIC: If Malignant -> Add to Open Cases
                        openCases.push({
                            id: patientId,
                            features: features,
                            date: new Date().toLocaleDateString(),
                            status: 'Malignant'
                        });
                        logDiv.innerHTML += `[${patientId}] -> MALIGNANT (Added to Open Cases)\n`;
                    } else {
                        // LOGIC: If Benign -> Auto-save to DB
                        logDiv.innerHTML += `[${patientId}] -> Benign (Archived to DB)\n`;
                    }

                } catch (error) {
                    logDiv.innerHTML += `[${patientId}] Error: ${error.message}\n`;
                }
                
                // Auto-scroll log
                logDiv.scrollTop = logDiv.scrollHeight;
            }

            logDiv.innerHTML += `\nAnalysis Complete. ${openCases.length} new cases require review.`;
        }

        // --- LOGIC: OPEN CASES ---
        function renderOpenCases() {
            const list = document.getElementById('cases-list');
            list.innerHTML = '';

            if (openCases.length === 0) {
                list.innerHTML = "<p style='text-align:center; color:#666;'>No open cases found.</p>";
                return;
            }

            openCases.forEach(c => {
                const item = document.createElement('div');
                item.className = 'case-card';
                item.innerHTML = `
                    <div>
                        <strong>ID: ${c.id}</strong> <span style="color:#999; font-size:0.9em">(${c.date})</span><br>
                        <span class="status-badge status-malignant">Potential Malignancy</span>
                    </div>
                    <div>
                        <button class="btn btn-success" onclick="resolveCase('${c.id}', 'Confirmed')">✓ Confirm</button>
                        <button class="btn btn-danger" onclick="resolveCase('${c.id}', 'False Positive')">X False Positive</button>
                    </div>
                `;
                list.appendChild(item);
            });
        }

        function resolveCase(id, resolution) {
            // Find the case data to send back to the backend
            const caseData = openCases.find(c => c.id === id);
            const features = caseData ? caseData.features : [];
            
            if(confirm(`Mark case ${id} as ${resolution}?`)) {
                
                // 1. Prepare the payload
                // We add 'operation': 'feedback' so the Lambda knows this isn't a new prediction
                const payload = {
                    operation: 'feedback',
                    id: id,
                    resolution: resolution
                };

                // 2. Send to API
                fetch(PREDICT_URL, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(payload)
                })
                .then(response => {
                    if (!response.ok) {
                        throw new Error(`Server responded with ${response.status}`);
                    }
                    return response.json();
                })
                .then(data => {
                    // 3. Success Handling
                    alert(`✅ Success! Feedback saved to DynamoDB.\nCase ${id} marked as ${resolution}.`);
                    
                    // Remove from the list
                    openCases = openCases.filter(c => c.id !== id);
                    renderOpenCases();
                })
                .catch(error => {
                    // 4. Error Handling (The "Why it didn't work" part)
                    console.error("Feedback Error:", error);
                    alert(`❌ Action Failed: Could not save to DynamoDB.\n\nReason: ${error.message}\n\nWhy is this happening?\n1. The Lambda function might not be updated yet to handle "feedback" requests.\n2. The DynamoDB table might be missing or named incorrectly.\n3. The API Gateway might be blocking the request (CORS).\n\nPlease check your Lambda logs for more details.`);
                });
            }
        }
    </script>
</head>
<body>

    <!-- HOME VIEW -->
    <div id="home-view" class="view active">
        <h1 style="text-align:center;">Breast Cancer Risk System</h1>
        
        <!-- Upload Section -->
        <div class="upload-area">
            <h3>📂 Upload Patient Data</h3>
            <p>Drag and drop your CSV file here or click to browse</p>
            <input type="file" id="csvFileInput" accept=".csv" onchange="handleFileUpload(event)" />
        </div>

        <!-- Data Preview Table -->
        <div id="table-container" class="table-container">
            <table id="data-table">
                <thead id="data-table-head"></thead>
                <tbody id="data-table-body"></tbody>
            </table>
        </div>

        <!-- Actions -->
        <div style="text-align:center; margin-top: 20px;">
            <button id="confirm-btn" class="btn btn-success" onclick="confirmData()" disabled>✓ Confirm Data</button>
            <button id="analyze-btn" class="btn btn-primary" onclick="analyzeRisk()" disabled>⚡ Analyze Risk</button>
            <button class="btn btn-outline" onclick="showView('cases-view')">📂 Open Cases</button>
        </div>
    </div>

    <!-- ANALYZE VIEW (Log Output) -->
    <div id="analyze-view" class="view">
        <button onclick="showView('home-view')" class="btn btn-outline">← Back to Home</button>
        <h2>Analysis in Progress</h2>
        
        <!-- MIRRORED TABLE -->
        <div id="analysis-table-container" class="table-container"></div>

        <p>Processing uploaded records against the AI model...</p>
        
        <div id="analysis-log"></div>
        
        <div style="margin-top: 20px; text-align: center;">
            <button class="btn btn-primary" onclick="showView('cases-view')">Go to Open Cases Review</button>
        </div>
    </div>

    <!-- OPEN CASES VIEW -->
    <div id="cases-view" class="view">
        <button onclick="showView('home-view')" class="btn btn-outline">← Back to Home</button>
        <h2>Open Cases</h2>
        <p>The following cases were flagged as <strong>Malignant</strong> and require doctor confirmation.</p>
        <div id="cases-list"></div>
    </div>

</body>
</html>

SyntaxError: invalid decimal literal (760329767.py, line 6)

# Phase 2: Continuous Learning (Retraining)
We will use the EC2 instance to retrain the model periodically.

### Step 9: Automate Retraining (Cron Job)
Since you already set up the environment and uploaded `train.py` in **Step 5**, you don't need to do it again. We just need to schedule the script to run automatically.

1.  **SSH into your EC2 instance** (if not connected).
2.  **Edit the Cron Table:**
    ```bash
    crontab -e
    ```
    *(Select `1` for `nano` if asked for an editor)*

3.  **Add the Schedule:**
    Scroll to the bottom of the file and add this line. This runs the training every Sunday at midnight and saves the output to a log file.
    ```bash
    0 0 * * 0 /home/ubuntu/micromamba/envs/breastcancer/bin/python /home/ubuntu/train.py >> /home/ubuntu/train.log 2>&1
    ```

4.  **Save and Exit:**
    *   Press `Ctrl+O`, `Enter` to save.
    *   Press `Ctrl+X` to exit.